# Final project: Meta Aggregation #

### by: Ortal & Raz & Michal
### guidence: Hila

In [ ]:
!pip install shap

In [ ]:
pip install scikit-multilearn

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from skmultilearn.problem_transform import ClassifierChain
from sklearn.model_selection import train_test_split
from GetProcessedData import get_question_dfs
from AggregationMethods.ConfidenceMethods import highest_average_confidence
from AggregationMethods.SurprisinglyPopular import surprisingly_pop_answer
from AggregationMethods.MajorityRule import majority_answer
from FeaturesExtraction.AnswerFeatures import AnswerF
from FeaturesExtraction.AnswerFeaturesSubgroups import AnswerSubF
from FeaturesExtraction.ConfidenceFeatures import ConfidenceF
from FeaturesExtraction.ConfidenceFeaturesSubgroups import ConfidenceSubF
from FeaturesExtraction.PredictionFeatures import PredictionsF
import Pipeline.MultiLabelPipeline as MLpip

In [2]:
#libraries
import sys
import os

module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# from ProjectFramework.GetProcessedData import get_question_dfs
# from ProjectFramework.Pipeline.MultiLabelPipeline import create_data_df

#import ProjectFramework.Pipeline.MultiLabelPipeline.MultiLabelPipeline.create_data_df
import numpy as np 
import pandas as pd 
import os
import json 
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import SGDClassifier

In [3]:
RF = RandomForestClassifier(random_state=0)
LR = LogisticRegression(solver = 'liblinear')
GNB = GaussianNB()
SVC = LinearSVC(random_state=0)
DT = DecisionTreeClassifier(random_state=0)
BNB = BernoulliNB()
SGD = SGDClassifier()

In [4]:
# get compleate data frame with features:

DATA = MLpip.get_data()
LABELS = MLpip.get_label_names()

y = DATA[LABELS]
x = DATA.drop(LABELS,axis='columns', inplace=False)

In [5]:
Accuracy = {}

In [ ]:
# split data into train and test using leave one out validation
results = {}
# res = []
loo = LeaveOneOut()
loo.get_n_splits(x)

for train_index, test_index in loo.split(x):
    # print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x.loc[train_index], x.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    # print(X_train, X_test, y_train, y_test)
    ## HERE - run moddels
    
#     clf_chain_RF = MLpip.classifier_chain(x_train, y_train, RandomForestClassifier(random_state=0))
#     results_RF.update(MLpip.get_model_results(clf_chain_RF, x_test))
#Accuracy['LR'] = MLpip.get_accuracy(results_RF, y)

    clf_chain_LR = MLpip.classifier_chain(x_train, y_train,LogisticRegression(solver = 'liblinear',max_iter=10000))
    results.update(MLpip.get_model_results(clf_chain_LR, x_test))
    
Accuracy['LR'] = MLpip.get_accuracy(results, y)

#     clf_chain_GNB = MLpip.classifier_chain(x_train, y_train, GaussianNB())
#     results.update(MLpip.get_model_results(clf_chain_GNB, x_test))
# Accuracy['GNB'] = MLpip.get_accuracy(results, y)



#     print("accuracy_score:", accuracy_score(y_test,res_svc))
#     print("Hamming_loss:", hamming_loss(y_test,res_svc))
#     print(metrics.classification_report(y_test, rounded_pred, zero_division=0))

In [7]:

Accuracy

{'GNB': 0.816793893129771}

In [ ]:
from sklearn.metrics import hamming_loss, accuracy_score 

print("accuracy_score:", accuracy_score(y, res))
print("Hamming_loss:", hamming_loss(y, res))

In [ ]:
###### split data into train and test using regular split

# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
print X_train.shape, y_train.shape
print X_test.shape, y_test.shape



In [ ]:
# split data into train and test using k-fold cross validation

kf = KFold(n_splits=2, random_state=None, shuffle=False) # Define the split - into 2 folds 
kf.get_n_splits(X) # returns the number of splitting iterations in the cross-validator

for train_index, test_index in kf.split(X):
    # print(“TRAIN:”, train_index, “TEST:”, test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    ## HERE - run moddels
    

In [ ]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics

# Perform k-fold cross validation
k = 10
scores = cross_val_score(model, x, y, cv=k)
print “Cross-validated scores:”, scores
    
# Make cross validated predictions
predictions = cross_val_predict(model, df, y, cv=k)
plt.scatter(y, predictions)

## present accuracy

In [ ]:
## present accuracy
accuracy_list = []

## add all models accuracy to list
accuracy_list.append(accuracy_score(nb_predict,y_test))
accuracy_list.append(accuracy_score(knn_predict,y_test))
accuracy_list.append(accuracy_score(rfc_predict,y_test))
accuracy_list.append(accuracy_score(logreg_predict,y_test))
accuracy_list.append(accuracy_score(svm_predict,y_test))
accuracy_list.append(accuracy_score(adc_predict,y_test))

accuracy_dataframe = pd.DataFrame(columns=["Model","Accuracy"])
model_list = ['Naive_Bayes','KNN','Random_Forest','Logistic_Reg','SVM','Ada_Boost']

for i in range(len(accuracy_list)):
    accuracy_dataframe.loc[i] = model_list[i], accuracy_list[i]

accuracy_dataframe

## shap feature evaluation

In [ ]:
# load JS vis
shap.initjs() 

## the best model:
model = rt_model

# create a tree explainer for Random Forest
explainer = shap.TreeExplainer(model)

#sample 100 observations from the test data
observations = X_test.sample(100, random_state=42)

#create an explainer
shap_values = explainer.shap_values(observations)

#generate a summary plot
shap.summary_plot(shap_values[1], features=observations)

## confusion matrix

In [ ]:
#confusion matrix visualization

def plot_confusion_matrix(test_Y, predict_y):
    C = confusion_matrix(test_Y, predict_y)
    A =(((C.T)/(C.sum(axis=1))).T)
    B =(C/C.sum(axis=0))
    plt.figure(figsize=(20,4))
    labels = [1,2]
    cmap=sns.light_palette("#45c6c4")
    plt.subplot(1, 3, 1)
    sns.heatmap(C, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Confusion matrix")
    plt.subplot(1, 3, 2)
    sns.heatmap(B, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Precision matrix")
    plt.subplot(1, 3, 3)
    sns.heatmap(A, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Recall matrix")
    plt.show()

plot_confusion_matrix(y_test, rfc_predict)